In [1]:
from sqlalchemy import create_engine 
from dotenv import load_dotenv
import os
import pandas as pd
import psycopg2

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")
if DATABASE_URL and DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)
    
engine = create_engine(DATABASE_URL, echo = False)


In [6]:
df = pd.read_csv("output.csv", index_col=0)

df.to_sql('data', con=engine, if_exists="append")


42

In [7]:
query = f"""SELECT * FROM data"""

In [8]:
import psycopg2

con = psycopg2.connect(DATABASE_URL)
cur = con.cursor()

In [39]:
test_data = pd.read_csv("output.csv")

In [48]:
test_data.head(1)

,category,title,link,organiser,date,day,time,address
0,VISUAL ARTS,BreatheWater,https://www.esplanade.com/festivals-and-series...,Parvathi Nayar (India),25 Mar – 24 Jul 2022,Fri – Sun,All Day,Theatre Street Cones


In [49]:
df.head(1)

,category,title,link,organiser,date,day,time,address
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [50]:
data_new = pd.concat([test_data, df], axis=0)

In [52]:
data_new.tail()

,category,title,link,organiser,date,day,time,address
38,THEATRE,Recipes for Wayfinding (Care Index) Installation,https://www.esplanade.com/festivals-and-series...,Alecia Neo (Singapore),28 Jul – 24 Sep 2022,Thu – Sat,All Day,Esplanade Roof Terrace
39,THEATRE,In Their Own Words: Writing Verbatim Plays,https://www.esplanade.com/festivals-and-series...,Haresh Sharma (Singapore),27-Aug-22,Sat,12pm,library@esplanade
40,THEATRE,Performing Grief,https://www.esplanade.com/festivals-and-series...,"Mohd Fared Jainal, Chong Tze Chien, Jolene Chi...",10-Sep-22,Sat,3pm,library@esplanade
41,THEATRE,In Conversation: And So The Conversations Cont...,https://www.esplanade.com/festivals-and-series...,Checkpoint Theatre (Singapore),24-Sep-22,Sat,3pm,library@esplanade
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [44]:
test = [] 
test.append({'category': "category_test",
                    'title': 'title_text',
                    'link': 'link_text',
                    'organiser': 'organiser_text',
                    'date': 'date_text',
                    'day': 'day_text',
                    'time': 'time_text',
                    'address': 'address_text'})

In [45]:
df = pd.DataFrame(test)

In [46]:
df

,category,title,link,organiser,date,day,time,address
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [21]:
def check_update(df, new_titles, con):
    query = f"""SELECT * FROM data"""
    old_title = list(pd.read_sql(query, con).loc[:,"title"])
    print("Number of old titles: {}".format(len(old_title)))
    for title in list(df["title"]):
        if title not in old_title:
            new_titles.append(title)
    print("Number of new new titles: {}".format(len(new_titles)))
    return new_titles

In [54]:
new_titles = []
update = check_update(data_new, new_titles, con)

c:\Users\joann\Anaconda3\envs\tensorflow-gpu\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Number of old titles: 42
Number of new new titles: 1


In [55]:
df_update = df[df["title"].isin(update)]

In [56]:
df_update

,category,title,link,organiser,date,day,time,address
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [ ]:
df_update.to_sql('data', con=engine, if_exists="append", index=False)